In [13]:
import pandas as pd
import os
import openai
import load_env_var
import json
import asyncio

In [14]:
# Import cleaned data
df = pd.read_csv('data/datos_actualizados.csv')

In [15]:
# Verifying the data
df.head(10)

,entity_name,city,state_abbreviation,variable_name,year,value,unit,definition
0,kcb bank,lakin,KS,total assets,2007,109409000.0,usd,the sum of all assets owned by the institution...
1,kcb bank,lakin,KS,total deposits,2007,86818000.0,usd,the sum of all deposits including demand depos...
2,kcb bank,lakin,KS,% insured (estimated),2007,0.0,percent,percent of deposits estimated to be insured. ...
3,kcb bank,lakin,KS,all real estate loans,2007,16477000.0,usd,"loans secured primarily by real estate, whethe..."
4,kcb bank,lakin,KS,total securities,2007,43618000.0,usd,total securities: the sum of held-to-maturity ...
5,mountain valley bank,dunlap,TN,total assets,1986,26478000.0,usd,the sum of all assets owned by the institution...
6,mountain valley bank,dunlap,TN,total deposits,1986,22363000.0,usd,the sum of all deposits including demand depos...
7,mountain valley bank,dunlap,TN,% insured (estimated),1986,0.0,percent,percent of deposits estimated to be insured. ...
8,mountain valley bank,dunlap,TN,all real estate loans,1986,6890000.0,usd,"loans secured primarily by real estate, whethe..."
9,mountain valley bank,dunlap,TN,total securities,1986,12222000.0,usd,total securities: the sum of held-to-maturity ...


### Embeddings Generation for Vector Search

In [16]:
# Load environment variables OpenAI
openai.api_type, openai.api_base, openai.api_version, openai.api_key, llm_model, emb_model = load_env_var.load_env_variables_openai()

In [17]:
def generate_embeddings(text, model=emb_model):
    response = openai.Embedding.create(input=text, engine=model)
    embeddings = response['data'][0]['embedding']
    return embeddings

### Embeddings Generation (Async to avoid rate limit)

In [18]:
async def generate_embeddings_async(text, model=emb_model):
    response = openai.Embedding.create(input=text, engine=model)
    embeddings = response['data'][0]['embedding']
    return embeddings

### Entity Name Column (async)

In [19]:
# List with unique entity names
unique_entity_name = df['entity_name'].unique()

In [20]:
id_unique_entity_name = list(range(1, len(unique_entity_name) + 1))

In [21]:
# Dataframe with unique entity names and their id
df_unique_entity_name = pd.DataFrame({'id_unique_entity_name': id_unique_entity_name, 'unique_entity_name': unique_entity_name})

In [22]:
# Checking the data
df_unique_entity_name.head()

,id_unique_entity_name,unique_entity_name
0,1,kcb bank
1,2,mountain valley bank
2,3,byron bank
3,4,banner banks
4,5,sallie mae bank


In [23]:
df_unique_entity_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798 entries, 0 to 2797
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id_unique_entity_name  2798 non-null   int64 
 1   unique_entity_name     2798 non-null   object
dtypes: int64(1), object(1)
memory usage: 43.8+ KB


In [24]:
async def write_json(row, category_vector):
    folder_path = 'data/json_entity_name/'
    
    file_name = os.path.join(folder_path, f'entity_name_{row["id_unique_entity_name"]}.json')
    
    content = {
        "id": f'entity_name_{row["id_unique_entity_name"]}',
        "category": "entity_name",
        "content": row["unique_entity_name"],
        "categoryVector": category_vector,
        "contentVector": await generate_embeddings_async(row["unique_entity_name"])
    }
    with open(file_name, "w") as file:
        json.dump(content, file)

In [25]:
async def main():
    category_vector = generate_embeddings("entity_name")
    
    for index, row in df_unique_entity_name.iterrows():
        await write_json(row, category_vector)
        await asyncio.sleep(3)

In [26]:
await main()

### City Column (async)

In [27]:
unique_city = df['city'].unique()

In [28]:
id_unique_city = list(range(1, len(unique_city) + 1))

In [29]:
df_unique_city = pd.DataFrame({'id_unique_city': id_unique_city, 'unique_city': unique_city})

In [30]:
df_unique_city.head()

,id_unique_city,unique_city
0,1,lakin
1,2,dunlap
2,3,byron
3,4,birnamwood
4,5,salt lake city


In [31]:
df_unique_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003 entries, 0 to 2002
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_unique_city  2003 non-null   int64 
 1   unique_city     2003 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [32]:
async def write_json(row, category_vector):
    folder_path = 'data/json_city/'
    
    file_name = os.path.join(folder_path, f'city_{row["id_unique_city"]}.json')
    
    content = {
        "id": f'city_{row["id_unique_city"]}',
        "category": "city",
        "content": row["unique_city"],
        "categoryVector": category_vector,
        "contentVector": await generate_embeddings_async(row["unique_city"])
    }
    with open(file_name, "w") as file:
        json.dump(content, file)

In [33]:
async def main():
    category_vector = generate_embeddings("city")
    
    for index, row in df_unique_city.iterrows():
        await write_json(row, category_vector)
        await asyncio.sleep(3)

In [34]:
await main()

### State Column

In [35]:
unique_state = df['state_abbreviation'].unique()

In [36]:
id_unique_state = list(range(1, len(unique_state) + 1))

In [37]:
df_unique_state = pd.DataFrame({'id_unique_state': id_unique_state, 'unique_state': unique_state})

In [38]:
df_unique_state.head()

,id_unique_state,unique_state
0,1,KS
1,2,TN
2,3,IL
3,4,WI
4,5,UT


In [39]:
df_unique_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_unique_state  54 non-null     int64 
 1   unique_state     54 non-null     object
dtypes: int64(1), object(1)
memory usage: 992.0+ bytes


In [40]:
folder_path = 'data/json_state/'

category_vector = generate_embeddings("state_abbreviation")

# Iterate over each row in the dataframe
for index, row in df_unique_state.iterrows():
    # Define the name of the file
    file_name = os.path.join(folder_path, f'state_{row["id_unique_state"]}.json')
    
    # Build a dictionary with the data
    content = {"id": f'state_{row["id_unique_state"]}', "category": "state_abbreviation", "content": row["unique_state"],
               "categoryVector": category_vector, "contentVector": generate_embeddings(row["unique_state"])}
    
    # Write the dictionary as a json file in the data folder
    with open(file_name, "w") as file:
        json.dump(content, file)

print('Files created successfully', folder_path)

Files created successfully data/json_state/


### Unit Column

In [41]:
unique_unit = df['unit'].unique()

In [42]:
id_unique_unit = list(range(1, len(unique_unit) + 1))

In [43]:
df_unique_unit = pd.DataFrame({'id_unique_unit': id_unique_unit, 'unique_unit': unique_unit})

In [44]:
df_unique_unit.head()

,id_unique_unit,unique_unit
0,1,usd
1,2,percent


In [45]:
df_unique_unit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_unique_unit  2 non-null      int64 
 1   unique_unit     2 non-null      object
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


In [46]:
folder_path = 'data/json_unit/'

# Iterate over each row in the dataframe
for index, row in df_unique_unit.iterrows():
    # Define the name of the file
    file_name = os.path.join(folder_path, f'unit_{row["id_unique_unit"]}.json')
    
    # Build a dictionary with the data
    content = {"id": f'unit_{row["id_unique_unit"]}', "category": "unit", "content": row["unique_unit"],
               "categoryVector": generate_embeddings("unit"), "contentVector": generate_embeddings(row["unique_unit"])}
    
    print(content)
    
    # Write the dictionary as a json file in the data folder
    with open(file_name, "w") as file:
        json.dump(content, file)

print('Files created successfully', folder_path)

{'id': 'unit_1', 'category': 'unit', 'content': 'usd', 'categoryVector': [0.014208773151040077, -0.0070002274587750435, 0.000839560350868851, 0.0090582724660635, -0.005369594786316156, 0.020573269575834274, 0.0036958614364266396, -0.032698854804039, -0.010595520958304405, -0.010875673964619637, -0.001234647585079074, 0.022426588460803032, -0.009834080003201962, 0.007671875413507223, -0.010868490673601627, 0.00255549605935812, 0.04074426740407944, 0.02771357260644436, 0.0163494274020195, -0.01744130440056324, 0.0010577562497928739, 0.002738672774285078, -0.009453359991312027, -0.009453359991312027, -0.010322551243007183, -0.0007690732018090785, 0.003911363426595926, -0.011486263014376163, 0.019093487411737442, -0.01384960301220417, 0.017412571236491203, -0.038014575839042664, -0.03071623668074608, -0.039250120520591736, -0.011493446305394173, 0.006529714446514845, 0.008411766029894352, 0.002866178285330534, 0.008304014801979065, -0.024825844913721085, 0.03338846191763878, 0.026693530380

### Variable Name and Definition

In [47]:
df_variable_name_def = df[['variable_name', 'definition']].head()

In [48]:
df_variable_name_def['id_variable_name_def'] = list(range(1, 6))

In [49]:
df_variable_name_def

,variable_name,definition,id_variable_name_def
0,total assets,the sum of all assets owned by the institution...,1
1,total deposits,the sum of all deposits including demand depos...,2
2,% insured (estimated),percent of deposits estimated to be insured. ...,3
3,all real estate loans,"loans secured primarily by real estate, whethe...",4
4,total securities,total securities: the sum of held-to-maturity ...,5


In [50]:
folder_path = 'data/json_var_name_def/'

category_vector = generate_embeddings("variable_name")

# Iterate over each row in the dataframe
for index, row in df_variable_name_def.iterrows():
    # Define the name of the file
    file_name = os.path.join(folder_path, f'var_name_def_{row["id_variable_name_def"]}.json')
    
    # Build a dictionary with the data
    content = {"id": f'var_name_def_{row["id_variable_name_def"]}', "category": "variable_name", "content": f'{row["variable_name"]}: {row["definition"]}',
               "categoryVector": category_vector, "contentVector": generate_embeddings(f'{row["variable_name"]}: {row["definition"]}')}
    
    # Write the dictionary as a json file in the data folder
    with open(file_name, "w") as file:
        json.dump(content, file)

print('Files created successfully', folder_path)

Files created successfully data/json_var_name_def/
